In [1]:
from fuse.dl.models.backbones.backbone_vit import vit_base
import torch


/u/liam/.conda/miniconda3/envs/kid/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
image_shape = [40,256,256]

backbone = vit_base(image_shape=image_shape, patch_shape=(8,32,32), channels=1)


In [11]:
input = torch.ones(image_shape).unsqueeze(0).unsqueeze(0)
backbone(input, pool='cls').shape

torch.Size([1, 768])

In [12]:
from fuse.dl.models.heads.heads_1D import Head1D
from fuse.dl.models import ModelMultiHead

conv_inputs = [("model.backbone_features", 768)]

heads=[
                # Head3DClassifier(
                #     head_name="head_0",
                #     # mode = "classification",
                #     conv_inputs=conv_inputs,
                #     dropout_rate=imaging_dropout,
                #     # num_outputs=num_classes,
                #     num_classes=num_classes,
                #     append_features=append_features,
                #     append_layers_description=(256, 128),
                #     fused_dropout_rate=fused_dropout,
                # ),
                Head1D(
                head_name="head_0",
                mode="classification",
                conv_inputs=conv_inputs,
                append_features = [("data.input.clinical.all", 11)],
                append_layers_description=(256, 128),
                num_outputs=2
                ),
            ]

model = ModelMultiHead(
        conv_inputs=(("data.input.img", 1),),
        backbone=backbone,
        heads=heads
    )